In [1]:
import re

import numpy as np
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA

from lxml import etree
from lxml import objectify

In [2]:
def read_data_gz(filename_str):
    
    # initiate parser
    parser = etree.XMLParser(ns_clean=True, encoding='utf-8')
    data_train_tree = etree.parse(filename_str,parser)

    # set up column names
    col_names = ['id','update-date','industries','job-name','salary','currency','employment','schedule','description','experience','contract']

    root = data_train_tree.getroot()  
    dt_train = pd.DataFrame(columns=col_names)
    y_train = pd.DataFrame()

    # process tree
    for k in range(len(root.getchildren())):
        dt_row = dict()   ## for storing rows
        dt_industr =  dict()  ## for storing industries
        if k % 100 ==0:
            print k
        ##k = 0
        b_childs = root.getchildren()[k].getchildren()
        dt_row['id'] = root.getchildren()[0].get("id")

        for tree_elem in b_childs:
            tag_val = tree_elem.tag
            num_val = tree_elem.text
            dt_row[tag_val] = num_val

            if tree_elem.tag == 'term':
                new_tag = tree_elem.getchildren()[0].tag
                dt_row[new_tag] = tree_elem.getchildren()[0].text

            if tree_elem.tag == 'requirement':
                new_tag = tree_elem.getchildren()[0].tag
                dt_row[new_tag] = tree_elem.getchildren()[0].text

            if tree_elem.tag == 'industries':
                for ind_elem in tree_elem.getchildren():
                    dt_industr [ind_elem.text] = 1

        dt_industr = pd.Series(dt_industr)
        dt_row = pd.Series(dt_row)

        ## print dt_industr
        ## print dt_row

        dt_train = dt_train.append(dt_row, ignore_index=True)
        y_train = y_train.append(dt_industr, ignore_index=True)
        
    return dt_train, y_train
    
dt_train, y_train = read_data_gz ("Data/train.xml.gz")
dt_test, y_test = read_data_gz ("Data/test.xml.gz")

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900


In [3]:
# joining all data
dt_all = pd.concat((dt_train,dt_test),axis=0)
dt_all.index = range (dt_all.shape[0])

print max(dt_all.index)

19999


In [4]:
## removing nans from y_train
y_train [np.isnan(y_train)] = 0

In [5]:
dt_all = dt_all.drop(['industries','requirement','term','currency','update-date'], axis=1)

In [6]:
# s - string salary
def process_salary(salary_str):
    l_bound = np.nan
    h_bound = np.nan
    try:
        m = re.search(u'от ([0-9]*)', salary_str)
        if m: 
            l_bound = float(m.group(1))
        m = re.search(u'до ([0-9]*)', salary_str)
        if m: 
            h_bound = float(m.group(1))
    except:
        pass
    return l_bound, h_bound  


salary_hl = pd.DataFrame(map(process_salary, dt_all['salary']),columns=['low_sal','high_sal'])

print dt_all.shape
print salary_hl.shape

dt_all = dt_all.join(salary_hl)
dt_all = dt_all.drop('salary', axis = 1)

(20000, 8)
(20000, 2)


In [7]:
## replacing nans with approximations
mean_rel = np.mean(dt_all['high_sal']/dt_all['low_sal'])

null_rows = np.isnan(dt_all['low_sal'])
dt_all.loc[null_rows,'low_sal'] = dt_all['high_sal'][null_rows] / mean_rel

null_rows = np.isnan(dt_all['high_sal'])
dt_all.loc[null_rows,'high_sal'] = dt_all['low_sal'][null_rows] * mean_rel


## replacing all missing values with zeroes
dt_all['no_sal_data'] = 0
null_rows = np.isnan(dt_all['high_sal'])
dt_all.loc[null_rows,'no_sal_data'] = 1
dt_all.loc[null_rows,'low_sal'] = 0
dt_all.loc[null_rows,'high_sal'] = 0

In [8]:
dt_all.head()

,id,job-name,employment,schedule,description,experience,contract,low_sal,high_sal,no_sal_data
0,174673,Бетонщик,Полная занятость,Полный день,<p>В Компанию с центральным офисом в Москве (в...,1-3 года,постоянный,0.0,0.0,1
1,174673,Руководитель интернет-проекта (обувь),Полная занятость,Полный день,<p><strong>Должностные обязанности:</strong></...,3-6 лет,постоянный,50000.0,70000.0,0
2,174673,Врач-офтальмолог,Полная занятость,Полный день,<p><strong>Обязанности:</strong></p> <ul> <li>...,3-6 лет,постоянный,0.0,0.0,1
3,174673,Специалист аналитического отдела,Проектная/Временная работа,Полный день,<p><strong>Обязанности:</strong></p> <ul> <li>...,1-3 года,постоянный,0.0,0.0,1
4,174673,Руководитель проекта лаборатории нефтехимическ...,Полная занятость,Полный день,<p>Руководитель проекта лаборатории нефтехимич...,1-3 года,постоянный,0.0,0.0,1


In [9]:
## Creating one-hot features for category columns

def make_bin_cols(table_data):
   pass 

col_names = ['employment','schedule','experience','contract']
dt_bin = pd.DataFrame()
dt = dt_all[col_names]
j = 0

for j in range(dt.shape[1]):
    for row_val in np.unique(dt.iloc[:,j]):
        new_col = np.zeros(dt.shape[0])   
        new_col[np.array(dt.iloc[:,j] == row_val)] = 1
        col_name = dt.columns.values[j] + "_" + row_val
        dt_bin[col_name] = new_col
        
dt_all = dt_all.join(dt_bin)
dt_all = dt_all.drop(col_names, axis =1 )
##pd.Series(np.unique(dt_all['employment']))

In [10]:
def clean_str(mystr):
    text = re.sub(u"<.*?>","", mystr)
    text = re.sub(u"&quot;","", text)
    return text
    
dt_all['description'] = map(clean_str, dt_all['description'])
dt_all['job-name'] = map(clean_str, dt_all['job-name'])


In [11]:
dt_all.head()

,id,job-name,description,low_sal,high_sal,no_sal_data,employment_Полная занятость,employment_Проектная/Временная работа,employment_Стажировка,employment_Частичная занятость,schedule_Вахтовый метод,schedule_Гибкий график,schedule_Полный день,schedule_Сменный график,schedule_Удаленная работа,experience_1-3 года,experience_3-6 лет,experience_Более 6 лет,experience_Нет опыта,contract_постоянный
0,174673,Бетонщик,В Компанию с центральным офисом в Москве (вход...,0.0,0.0,1,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
1,174673,Руководитель интернет-проекта (обувь),Должностные обязанности: Управление ИНТЕРНЕТ ...,50000.0,70000.0,0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
2,174673,Врач-офтальмолог,Обязанности: проведение полной диагностики со...,0.0,0.0,1,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
3,174673,Специалист аналитического отдела,Обязанности: Обработка больших массивов инфор...,0.0,0.0,1,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
4,174673,Руководитель проекта лаборатории нефтехимическ...,Руководитель проекта лаборатории нефтехимическ...,0.0,0.0,1,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0


In [ ]:
##  nltk.download("punkt")
##
##  http://www.markhneedham.com/blog/2015/02/15/pythonscikit-learn-calculating-tfidf-on-how-i-met-your-mother-transcripts/
##

import nltk
import string

from nltk.stem.snowball import SnowballStemmer

def tokenize(text):
    stemmer = SnowballStemmer("russian")
    tokens = nltk.word_tokenize(text)
    stems = []
    for item in tokens:
        stems.append(stemmer.stem(item))
    return stems


tfidf = TfidfVectorizer(tokenizer=tokenize, ngram_range=(1, 5))
tfs_train = tfidf.fit_transform(dt_all['description'][0:9999])
tfs_test  = tfidf.fit_transform(dt_all['description'][10000:19999])

##  text = dt_all['description'][0]
##  print pd.Series(tokenize (text))
##  text = text.lower()
##  text = text.translate(None, string.punctuation)
##  tokens = nltk.word_tokenize(text)
##  print (tokens)

In [ ]:
pca_tf = PCA(n_components = 200)
tfs_train = pca_tf.fit_transform (tfs_train.todense())
tfs_test  = pca_tf.transform (tfs_test.todense())


In [ ]:
tfs_train[0:100,:]

In [ ]:
feature_names = tf.get_feature_names() 
